# Step 2: Combine staffing levels

This file combines all staffing level files in the EDSIGHT_STAFFING folder, and generates two files, described below:

### output/2_staffing.csv

This file contains aggregate annual staffing totals by district and year. It includes only public school districts, and it contains a column denoting whether that year contained any null values -- meaning categories of employees where no numeric value was provided. This appears to represent legitimate zero values for many rows, but it also appears to indicate missing data in some cases. The reason we need this "asterisk" is that in cases where data is missing, it would falsely be identified later on as a cut or increase.


### output/byproduct_2_all_staffing.csv

This file is not aggregated, and is not limited to public school districts. It is NOT used in the workflow, and therefore not prefixed with a number. It is only saved as output because it seems like a pretty useful thing to have.



### ./data files

Here's what's in the data folder

In [16]:
ls "data/EDSIGHT_STAFFING/"

FTEStaffing (1).csv   FTEStaffing (15).csv  FTEStaffing (6).csv
FTEStaffing (10).csv  FTEStaffing (16).csv  FTEStaffing (7).csv
FTEStaffing (11).csv  FTEStaffing (2).csv   FTEStaffing (8).csv
FTEStaffing (12).csv  FTEStaffing (3).csv   FTEStaffing (9).csv
FTEStaffing (13).csv  FTEStaffing (4).csv   FTEStaffing.csv
FTEStaffing (14).csv  FTEStaffing (5).csv


In [17]:
import pandas as pd

In [18]:
class EdsightReport(object):
    
    def fix_equals(self, col):
        self.df[col] = self.df[col].str.strip("=").str.strip("\"")
    
    def __init__(self, fname):
        self.fname = fname
        self.report_name = open(self.fname).readline().strip().strip("\"")
        self.df = pd.read_csv(self.fname,skiprows=2)

In [19]:
class SchoolOrganizations(EdsightReport):
    
    def __init__(self, *args, **kwargs):
        super(SchoolOrganizations, self).__init__("data/findSchoolDistrict.csv", *args, **kwargs)
        self.fix_equals("OrganizationCode")
        self.fix_equals("ZIP")
        for grade_column in ["Grade " + str(x) for x in range(1, 13)]:
            self.fix_equals(grade_column)
            
class Districts(SchoolOrganizations):
    
    def find_by_city(self, city):
        return self.df[self.df["City"].str.upper().str.strip().str.contains(city.upper().strip())][["OrganizationName","OrganizationType"]]
    
    def find_by_name(self, name):
        return self.df[self.df["OrganizationName"].str.upper() == name.upper()]
    
    def __init__(self, *args, **kwargs):
        super(Districts, self).__init__()
        self.df = self.df[self.df["OrganizationType"].str.contains("District")]
        
    
districts = Districts()
print districts.report_name
districts.df["OrganizationType"].value_counts()
print districts.find_by_city("Bridgeport")

Find School/District Results
                                     OrganizationName  \
719                        Bridgeport School District   
807   Capital Preparatory Harbor School Inc. District   
1135               Great Oaks Charter School District   
1493       New Beginnings Inc Family Academy District   
1730           Park City Prep Charter School District   
2057                      The Bridge Academy District   

                     OrganizationType  
719           Public School Districts  
807   Public Charter School Districts  
1135  Public Charter School Districts  
1493  Public Charter School Districts  
1730  Public Charter School Districts  
2057  Public Charter School Districts  


In [20]:
class StaffingReport(EdsightReport):
    
    def __init__(self, *args, **kwargs):
        super(StaffingReport, self).__init__(*args, **kwargs)
        self.year = self.report_name.replace("FTE Staffing Report for ", "")
        self.fiscal_year = self.year[:2] + self.year[-2:]
        self.df["District"] = self.df["District"].fillna(method="ffill")
        self.fix_equals("District")
        self.fix_equals("Assignment Category")
        self.fix_equals("Educator Type")
        self.df = self.df.set_index(["District","Assignment Category","Educator Type"])
        self.df["report_year"] = self.fiscal_year
        self.df = self.df.reset_index().set_index("District").join(districts.df.set_index("OrganizationName")[["City","OrganizationCode","OrganizationType"]])

er = StaffingReport("data/EDSIGHT_STAFFING/FTEStaffing (1).csv")
print er.report_name
print er.year
print er.fiscal_year
er.df.head()


FTE Staffing Report for 2002-03
2002-03
2003


,Assignment Category,Educator Type,FTE Count,report_year,City,OrganizationCode,OrganizationType
Amistad Academy District,Administrators Coordinators and Department Ch...,Certified,1.8,2003,New Haven,2790013,Public Charter School Districts
Amistad Academy District,Counselors Social Workers and School Psychol...,Certified,1.0,2003,New Haven,2790013,Public Charter School Districts
Amistad Academy District,General Education - Teachers and Instructors,Certified,14.9,2003,New Haven,2790013,Public Charter School Districts
Amistad Academy District,General Education - Paraprofessional Instructi...,Non-Certified,3.0,2003,New Haven,2790013,Public Charter School Districts
Amistad Academy District,Library/Media - Support Staff,Non-Certified,0.0,2003,New Haven,2790013,Public Charter School Districts


In [21]:
print "These are districts that can't be reconciled to a city"
set(er.df[er.df["City"].isnull()].index.tolist())

These are districts that can't be reconciled to a city


{'Highville Mustard Seed Charter School District',
 'Office of Early Childhood (OEC)',
 'Project Oceanology',
 'Unified School District #1',
 'Unified School District #2'}

In [22]:
# Now pull in all the school stuff
import os

def all_staffing_years():
    frames = []
    years = []

    for f in os.listdir("data/EDSIGHT_STAFFING/"):
        fpath = os.path.join("data/EDSIGHT_STAFFING/",f)
        report = StaffingReport(fpath)
        #print (report.fiscal_year)
        if int(report.fiscal_year) in years:
            raise Exception("Duplicate year: " + str(report.fiscal_year))
        years.append(int(report.fiscal_year))
        new_df = report.df.reset_index()
        new_df["report_year"] = new_df["report_year"].astype(int)
        new_df = new_df.set_index(["report_year","index","Assignment Category","Educator Type"])
        frames.append(new_df)
    years.sort()
    print years
    ret =  pd.concat(frames)
    def fix_city(x):
        try:
            return x.strip().upper()
        except:
            return x
    ret["City"] = ret["City"].apply(fix_city)
    return ret

staffing_df = all_staffing_years()
staffing_df.head()

[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


FTE Count  \
report_year index                    Assignment Category                                Educator Type              
2003        Amistad Academy District Administrators  Coordinators and Department Cha... Certified            1.8   
                                     Counselors  Social Workers  and School Psycholo... Certified            1.0   
                                     General Education - Teachers and Instructors       Certified           14.9   
                                     General Education - Paraprofessional Instructio... Non-Certified        3.0   
                                     Library/Media - Support Staff                      Non-Certified        0.0   

                                                                                                            City  \
report_year index                    Assignment Category                                Educator Type              
2003        Amistad Academy District Administrators  Coordinators and Department Cha... Certified      NEW HAVEN   
                                     Counselors  Social Workers  and School Psycholo... Certified      NEW HAVEN   
                                     General Education - Teachers and Instructors       Certified      NEW HAVEN   
                                     General Education - Paraprofessional Instructio... Non-Certified  NEW HAVEN   
                                     Library/Media - Support Staff                      Non-Certified  NEW HAVEN   

                                                                                                      OrganizationCode  \
report_year index                    Assignment Category                                Educator Type                    
2003        Amistad Academy District Administrators  Coordinators and Department Cha... Certified              2790013   
                                     Counselors  Social Workers  and School Psycholo... Certified              2790013   
                                     General Education - Teachers and Instructors       Certified              2790013   
                                     General Education - Paraprofessional Instructio... Non-Certified          2790013   
                                     Library/Media - Support Staff                      Non-Certified          2790013   

                                                                                                                      OrganizationType  
report_year index                    Assignment Category                                Educator Type                                   
2003        Amistad Academy District Administrators  Coordinators and Department Cha... Certified      Public Charter School Districts  
                                     Counselors  Social Workers  and School Psycholo... Certified      Public Charter School Districts  
                                     General Education - Teachers and Instructors       Certified      Public Charter School Districts  
                                     General Education - Paraprofessional Instructio... Non-Certified  Public Charter School Districts  
                                     Library/Media - Support Staff                      Non-Certified  Public Charter School Districts

In [23]:
staffing_df.to_csv("output/byproduct_2_all_staffing.csv")

## What types of educators are there?

In [24]:
staffing_df.reset_index()["Assignment Category"].value_counts()

Other Staff Providing Non-Instructional Services/Support                        3276
Special Education - Paraprofessional Instructional Assistants                   3260
General Education - Paraprofessional Instructional Assistants                   3260
School Nurses                                                                   3260
Library/Media - Support Staff                                                   3260
General Education - Teachers and Instructors                                    3147
Administrators  Coordinators and Department Chairs - School Level               3087
Special Education - Teachers and Instructors                                    3064
Counselors  Social Workers  and School Psychologists                            2948
Administrators  Coordinators and Department Chairs - District Central Office    2829
Instructional Specialists Who Support Teachers                                  2794
Library/Media - Specialists (Certified)                          

# Drop non-public districsts

In [25]:

# Filter non-public schools
staffing_df = staffing_df[staffing_df["OrganizationType"]=="Public School Districts"]
print("Should only have one type of organization...")
print (staffing_df["OrganizationType"].value_counts())


Should only have one type of organization...
Public School Districts    28828
Name: OrganizationType, dtype: int64


# Add a has_nulls column so we know which years of data contain some null values

In [26]:
# See how many categories are empty
check_empties = pd.pivot_table(staffing_df.reset_index(),
                   index=["City","report_year"],
                   values="FTE Count",
                   columns="Assignment Category")
check_empties.head()

Assignment Category  Administrators  Coordinators and Department Chairs - District Central Office  \
City    report_year                                                                                 
ANDOVER 2002                                                       NaN                              
        2003                                                       0.6                              
        2004                                                       0.6                              
        2005                                                       0.3                              
        2006                                                       0.3                              

Assignment Category  Administrators  Coordinators and Department Chairs - School Level  \
City    report_year                                                                      
ANDOVER 2002                                                       NaN                   
        2003                                                       1.4                   
        2004                                                       1.4                   
        2005                                                       2.0                   
        2006                                                       2.0                   

Assignment Category  Counselors  Social Workers  and School Psychologists  \
City    report_year                                                         
ANDOVER 2002                                                       NaN      
        2003                                                       NaN      
        2004                                                       NaN      
        2005                                                       NaN      
        2006                                                       NaN      

Assignment Category  General Education - Paraprofessional Instructional Assistants  \
City    report_year                                                                  
ANDOVER 2002                                                       4.1               
        2003                                                       6.3               
        2004                                                       7.5               
        2005                                                       3.2               
        2006                                                       2.7               

Assignment Category  General Education - Teachers and Instructors  \
City    report_year                                                 
ANDOVER 2002                                                  NaN   
        2003                                                 23.6   
        2004                                                 23.8   
        2005                                                 25.8   
        2006                                                 24.8   

Assignment Category  Instructional Specialists Who Support Teachers  \
City    report_year                                                   
ANDOVER 2002                                                    NaN   
        2003                                                    2.0   
        2004                                                    2.0   
        2005                                                    1.0   
        2006                                                    2.0   

Assignment Category  Library/Media - Specialists (Certified)  \
City    report_year                                            
ANDOVER 2002                                             NaN   
        2003                                             1.0   
        2004                                             1.0   
        2005                                             1.0   
        2006                                             1.0   

Assignment Category  Library/Media - Support Staff  \
City    report_year              

In [27]:
check_empties[check_empties["General Education - Teachers and Instructors"].isnull()]

,Assignment Category,Administrators Coordinators and Department Chairs - District Central Office,Administrators Coordinators and Department Chairs - School Level,Counselors Social Workers and School Psychologists,General Education - Paraprofessional Instructional Assistants,General Education - Teachers and Instructors,Instructional Specialists Who Support Teachers,Library/Media - Specialists (Certified),Library/Media - Support Staff,Other Staff Providing Non-Instructional Services/Support,School Nurses,Special Education - Paraprofessional Instructional Assistants,Special Education - Teachers and Instructors
City,report_year,,,,,,,,,,,,
ANDOVER,2002,NaN,NaN,NaN,4.1,NaN,NaN,NaN,0.0,18.0,1.0,7.2,NaN
ANSONIA,2002,NaN,NaN,NaN,23.1,NaN,NaN,NaN,5.0,61.7,4.0,18.3,NaN
ASHFORD,2002,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,11.0,1.2,7.5,NaN
AVON,2002,NaN,NaN,NaN,8.5,NaN,NaN,NaN,6.0,99.0,4.3,23.0,NaN
BALTIC,2002,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.0,7.8,1.3,8.0,NaN
BARKHAMSTED,2002,NaN,NaN,NaN,7.9,NaN,NaN,NaN,0.0,7.7,1.0,0.0,NaN
BERLIN,2002,NaN,NaN,NaN,14.9,NaN,NaN,NaN,5.0,54.1,6.0,39.0,NaN
BETHANY,2002,NaN,NaN,NaN,6.5,NaN,NaN,NaN,1.0,9.0,1.0,4.0,NaN
BETHEL,2002,NaN,NaN,NaN,14.0,NaN,NaN,NaN,6.0,52.0,7.0,41.0,NaN


# Wow, let's just forget about 2002. we don't neee it anyway

See table above...

In [28]:
print len(check_empties)
print len(check_empties.dropna(how="any"))
print ("This %d should equal..." % (len(check_empties) - len(check_empties.dropna(how="any"))))

2482
2017
This 465 should equal...


In [29]:
print ("... this: %d" % len(check_empties[check_empties.isnull().any(axis=1)]))

... this: 465


In [30]:
try:
    assert(len(check_empties) - len(check_empties.dropna(how="any")) ==\
                   len(check_empties[check_empties.isnull().any(axis=1)]))
except:
    raise Exception("ERROR! ERROR! These values should match!")

In [31]:
any_empties = check_empties[check_empties.isnull().any(axis=1)]
any_empties["has_nulls"] = True
any_empties["has_nulls"].to_frame().to_csv("output/2_has_nulls.csv")
any_empties["has_nulls"].to_frame()

/Users/jakekara/pandas/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


has_nulls
City          report_year           
ANDOVER       2002              True
              2003              True
              2004              True
              2005              True
              2006              True
              2007              True
              2008              True
              2009              True
              2010              True
              2011              True
              2012              True
              2013              True
              2014              True
              2015              True
              2016              True
              2017              True
              2018              True
ANSONIA       2002              True
ASHFORD       2002              True
              2003              True
              2004              True
              2005              True
              2006              True
              2007              True
              2008              True
              2009              True
              2010              True
              2011              True
              2012              True
              2013              True
...                              ...
WEST HAVEN    2002              True
WESTBROOK     2002              True
WESTON        2002              True
              2006              True
WESTPORT      2002              True
WETHERSFIELD  2002              True
WILLIMANTIC   2002              True
WILLINGTON    2002              True
              2003              True
              2004              True
              2005              True
              2006              True
WILTON        2002              True
WINDSOR       2002              True
WINDSOR LOCKS 2002              True
WINSTED       2002              True
              2004              True
              2005              True
              2006              True
              2007              True
              2009              True
              2010              True
              2011              True
              2012              True
              2013              True
WOLCOTT       2002              True
              2010              True
WOODBRIDGE    2002              True
              2015              True
WOODSTOCK     2002              True

[465 rows x 1 columns]

# How many cities have more than one district?

Only Easton and Deep River. If they come up in our analysis, we'll look closer.

In [32]:
# only easton and deep river
staffing_df.reset_index().groupby("City").agg({"index":pd.Series.nunique}).sort_values(by="index",ascending=False).head()

,index
City,
DEEP RIVER,3
EASTON,2
ANDOVER,1
RIDGEFIELD,1
PLAINFIELD,1


# Get totals by city

In [33]:
staffing_df_copy = staffing_df.copy()
def limit_educator_type(assignment):
    return staffing_df_copy[staffing_df.index.map(
        lambda x: x[2] == assignment
    )]

teacher_df = limit_educator_type("General Education - Teachers and Instructors")
teacher_df
                        

FTE Count  \
report_year index                         Assignment Category                          Educator Type              
2003        Andover School District       General Education - Teachers and Instructors Certified           23.6   
            Ansonia School District       General Education - Teachers and Instructors Certified          136.8   
            Ashford School District       General Education - Teachers and Instructors Certified           37.0   
            Avon School District          General Education - Teachers and Instructors Certified          198.5   
            Barkhamsted School District   General Education - Teachers and Instructors Certified           20.1   
            Berlin School District        General Education - Teachers and Instructors Certified          197.4   
            Bethany School District       General Education - Teachers and Instructors Certified           39.8   
            Bethel School District        General Education - Teachers and Instructors Certified          214.7   
            Bloomfield School District    General Education - Teachers and Instructors Certified          166.0   
            Bolton School District        General Education - Teachers and Instructors Certified           73.5   
            Bozrah School District        General Education - Teachers and Instructors Certified           19.7   
            Branford School District      General Education - Teachers and Instructors Certified          239.8   
            Bridgeport School District    General Education - Teachers and Instructors Certified         1243.6   
            Bristol School District       General Education - Teachers and Instructors Certified          528.8   
            Brookfield School District    General Education - Teachers and Instructors Certified          198.4   
            Brooklyn School District      General Education - Teachers and Instructors Certified           61.0   
            Canaan School District        General Education - Teachers and Instructors Certified           12.3   
            Canterbury School District    General Education - Teachers and Instructors Certified           39.0   
            Canton School District        General Education - Teachers and Instructors Certified          101.8   
            Chaplin School District       General Education - Teachers and Instructors Certified           20.4   
            Cheshire School District      General Education - Teachers and Instructors Certified          294.4   
            Chester School District       General Education - Teachers and Instructors Certified           23.9   
            Clinton School District       General Education - Teachers and Instructors Certified          148.0   
            Colchester School District    General Education - Teachers and Instructors Certified          195.7   
            Colebrook School District     General Education - Teachers and Instructors Certified            9.5   
            Columbia School District      General Education - Teachers and Instructors Certified           49.8   
            Cornwall School District      General Education - Teachers and Instructors Certified           16.5   
            Coventry School District      General Education - Teachers and Instructors Certified          129.5   
            Cromwell School District      General Education - Teachers and Instructors Certified          126.7   
            Danbury School District       General Education - Teachers and Instructors Certified          572.6   
...                                                                                                         ...   
2016        Stonington School District    General Education - Teachers and Instructors Certified          145.1   
            Stratford School District     General Education - Teachers and Instructors Certified          429.3   
            Suffield School District      General Education - Teachers and In

In [34]:
counselor_df = limit_educator_type("Counselors  Social Workers  and School Psychologists")
counselor_df

FTE Count  \
report_year index                         Assignment Category                                Educator Type              
2003        Ansonia School District       Counselors  Social Workers  and School Psycholo... Certified            9.0   
            Ashford School District       Counselors  Social Workers  and School Psycholo... Certified            1.0   
            Avon School District          Counselors  Social Workers  and School Psycholo... Certified           15.2   
            Barkhamsted School District   Counselors  Social Workers  and School Psycholo... Certified            1.3   
            Berlin School District        Counselors  Social Workers  and School Psycholo... Certified           16.8   
            Bethany School District       Counselors  Social Workers  and School Psycholo... Certified            2.4   
            Bethel School District        Counselors  Social Workers  and School Psycholo... Certified           18.3   
            Bloomfield School District    Counselors  Social Workers  and School Psycholo... Certified           15.6   
            Bolton School District        Counselors  Social Workers  and School Psycholo... Certified            5.4   
            Bozrah School District        Counselors  Social Workers  and School Psycholo... Certified            0.5   
            Branford School District      Counselors  Social Workers  and School Psycholo... Certified           20.3   
            Bridgeport School District    Counselors  Social Workers  and School Psycholo... Certified          111.2   
            Bristol School District       Counselors  Social Workers  and School Psycholo... Certified           33.3   
            Brookfield School District    Counselors  Social Workers  and School Psycholo... Certified           12.7   
            Brooklyn School District      Counselors  Social Workers  and School Psycholo... Certified            4.0   
            Canaan School District        Counselors  Social Workers  and School Psycholo... Certified            0.5   
            Canterbury School District    Counselors  Social Workers  and School Psycholo... Certified            3.0   
            Canton School District        Counselors  Social Workers  and School Psycholo... Certified            8.5   
            Chaplin School District       Counselors  Social Workers  and School Psycholo... Certified            1.0   
            Cheshire School District      Counselors  Social Workers  and School Psycholo... Certified           22.5   
            Chester School District       Counselors  Social Workers  and School Psycholo... Certified            1.0   
            Clinton School District       Counselors  Social Workers  and School Psycholo... Certified           12.0   
            Colchester School District    Counselors  Social Workers  and School Psycholo... Certified           13.4   
            Colebrook School District     Counselors  Social Workers  and School Psycholo... Certified            0.4   
            Columbia School District      Counselors  Social Workers  and School Psycholo... Certified            3.0   
            Cornwall School District      Counselors  Social Workers  and School Psycholo... Certified            0.5   
            Coventry School District      Counselors  Social Workers  and School Psycholo... Certified           10.6   
            Cromwell School District      Counselors  Social Workers  and School Psycholo... Certified            7.0   
            Danbury School District       Counselors  Social Workers  and School Psycholo... Certified           49.6   
            Darien School District        Counselors  Social Workers  and School Psycholo... Certified           20.6   
...                                                                                                               ...   
2016        Sterling School District      Counselors  Social Workers  and School Psycholo... Certified            1

In [35]:
staffing_df.index.map(lambda x: x[2]).value_counts()

Other Staff Providing Non-Instructional Services/Support                        2511
Special Education - Paraprofessional Instructional Assistants                   2510
General Education - Paraprofessional Instructional Assistants                   2510
School Nurses                                                                   2510
Library/Media - Support Staff                                                   2510
General Education - Teachers and Instructors                                    2384
Special Education - Teachers and Instructors                                    2376
Administrators  Coordinators and Department Chairs - School Level               2374
Administrators  Coordinators and Department Chairs - District Central Office    2337
Counselors  Social Workers  and School Psychologists                            2332
Instructional Specialists Who Support Teachers                                  2322
Library/Media - Specialists (Certified)                          

## Get and combine totals

In [36]:
staffing_df = staffing_df.groupby(["City","report_year"]).agg({"FTE Count":sum})
staffing_df.head()

FTE Count
City    report_year           
ANDOVER 2002              30.3
        2003              62.2
        2004              62.2
        2005              60.6
        2006              59.6

In [37]:
counselor_df = counselor_df.groupby(["City","report_year"]).agg({"FTE Count":sum})
counselor_df.columns = ["Counselor FTE Count"]
counselor_df.head()

Counselor FTE Count
City    report_year                     
ANSONIA 2003                         9.0
        2004                        12.0
        2005                        12.0
        2006                        12.0
        2007                        12.0

In [38]:
teacher_df = teacher_df.groupby(["City","report_year"]).agg({"FTE Count":sum})
teacher_df.columns = ["Teacher FTE Count"]
teacher_df.head()

Teacher FTE Count
City    report_year                   
ANDOVER 2003                      23.6
        2004                      23.8
        2005                      25.8
        2006                      24.8
        2007                      23.8

In [39]:
staffing_df = staffing_df.join(teacher_df).join(counselor_df)

In [40]:
# staffing_df = staffing_df.groupby(["City","report_year"]).agg({"FTE Count":sum})
# staffing_df.head()

In [41]:
any_empties

Assignment Category        Administrators  Coordinators and Department Chairs - District Central Office  \
City          report_year                                                                                 
ANDOVER       2002                                                       NaN                              
              2003                                                       0.6                              
              2004                                                       0.6                              
              2005                                                       0.3                              
              2006                                                       0.3                              
              2007                                                       0.3                              
              2008                                                       0.3                              
              2009                                                       0.3                              
              2010                                                       0.3                              
              2011                                                       0.3                              
              2012                                                       0.6                              
              2013                                                       0.6                              
              2014                                                       0.6                              
              2015                                                       0.6                              
              2016                                                       2.6                              
              2017                                                       2.6                              
              2018                                                       2.1                              
ANSONIA       2002                                                       NaN                              
ASHFORD       2002                                                       NaN                              
              2003                                                       0.8                              
              2004                                                       0.8                              
              2005                                                       NaN                              
              2006                                                       1.0                              
              2007                                                       1.0                              
              2008                                                       NaN                              
              2009                                                       NaN                              
              2010                                                       NaN                              
              2011                                                       NaN                              
              2012                                                       NaN                              
              2013                                                       NaN                              
...                                                                      ...                              
WEST HAVEN    2002                                                       NaN                              
WESTBROOK     2002                                                       NaN                              
WESTON        2002                                                       NaN                              
              2006                                                       3.0                              
WESTPORT      2002                       

In [57]:
staffing_df.loc["WINDHAM"]

KeyError: u'the label [WINDHAM] is not in the [index]'

# Add asterisks column

In [43]:
staffing_df = staffing_df.join(any_empties[["has_nulls"]])#.to_frame())
#staffing_df.join(any_empties[["has_nulls"]])#.to_frame())


In [44]:
staffing_df["has_nulls"] = staffing_df["has_nulls"].fillna("False")
staffing_df

FTE Count  Teacher FTE Count  Counselor FTE Count  \
City       report_year                                                      
ANDOVER    2002              30.3                NaN                  NaN   
           2003              62.2               23.6                  NaN   
           2004              62.2               23.8                  NaN   
           2005              60.6               25.8                  NaN   
           2006              59.6               24.8                  NaN   
           2007              60.9               23.8                  NaN   
           2008              58.0               25.3                  NaN   
           2009              59.4               25.3                  NaN   
           2010              60.0               26.3                  NaN   
           2011              59.7               26.3                  NaN   
           2012              57.6               25.2                  NaN   
           2013              56.6               24.7                  NaN   
           2014              56.6               24.7                  NaN   
           2015              52.1               19.7                  NaN   
           2016              54.3               22.8                  NaN   
           2017              45.3               17.8                  NaN   
           2018              46.3               19.8                  NaN   
ANSONIA    2002             112.1                NaN                  NaN   
           2003             314.2              136.8                  9.0   
           2004             346.9              141.5                 12.0   
           2005             363.1              150.7                 12.0   
           2006             368.7              146.0                 12.0   
           2007             489.8              147.0                 12.0   
           2008             391.9              148.0                 12.0   
           2009             404.5              152.0                 13.6   
           2010             373.2              143.8                 13.6   
           2011             399.6              147.0                 13.6   
           2012             389.6              142.0                 13.6   
           2013             376.7              139.6                 13.6   
           2014             391.7              145.0                 13.0   
...                           ...                ...                  ...   
WOODBRIDGE 2006             137.7               58.5                  3.5   
           2007             134.3               58.5                  2.5   
           2008             139.2               59.5                  2.5   
           2009             139.2               57.5                  2.5   
           2010             136.2               56.5                  2.5   
           2011             130.2               56.5                  2.5   
           2012             130.7               58.0                  2.5   
           2013             127.2               58.0                  2.5   
           2014             128.2               59.0                  2.5   
           2015              81.5               61.0                  2.5   
           2016             131.6               58.0                  2.5   
           2017             133.1               59.5                  2.5   
           2018             131.6               60.0                  2.5   
WOODSTOCK  2002              73.0                NaN                  NaN   
           2003             163.7               60.0                  2.4   
           2004             162.9               60.0                  3.4   
           2005             152.7               58.0                  3.4   
           2006             158.7               60.0                  3.4   
           2007             154.2               57.0                  3.4   
           2008     

In [45]:
staffing_df.to_csv("output/2_staffing.csv")

In [46]:
staffing_df[staffing_df["has_nulls"] == True].reset_index()["report_year"].value_counts()

2002    146
2017     28
2016     28
2018     25
2012     21
2015     21
2005     21
2013     20
2006     19
2014     18
2004     18
2003     18
2010     18
2007     17
2011     16
2008     16
2009     15
Name: report_year, dtype: int64

In [47]:
staffing_df = staffing_df[staffing_df.index.map(lambda x: x[1] > 2004)]

In [48]:
staffing_df[staffing_df["has_nulls"] == True].reset_index()["report_year"].value_counts().to_frame().sort_index()

,report_year
2005,21
2006,19
2007,17
2008,16
2009,15
2010,18
2011,16
2012,21
2013,20
2014,18


In [49]:
staffing_df[staffing_df["has_nulls"] == True].reset_index()["City"].value_counts()

CHAPLIN            14
CANTERBURY         14
NORTH FRANKLIN     14
BOZRAH             14
ANDOVER            14
ASHFORD            14
BALTIC             14
COLEBROOK          14
UNION              14
BETHANY            12
EASTFORD           11
SCOTLAND           11
CORNWALL            9
VOLUNTOWN           8
WINSTED             8
CANAAN              7
LISBON              7
POMFRET CENTER      6
HAMPTON             6
EAST HARTLAND       5
BARKHAMSTED         5
STERLING            5
LAKEVILLE           5
DERBY               4
NORFOLK             4
FALLS VILLAGE       4
NORWALK             3
SHARON              3
TERRYVILLE          3
EAST WINDSOR        3
NEW LONDON          3
PRESTON             3
BROOKLYN            2
HEBRON              2
KENT                2
WILLINGTON          2
SALEM               2
EASTON              2
BOLTON              1
WESTON              1
SOMERS              1
MARLBOROUGH         1
WOODBRIDGE          1
BRANFORD            1
COVENTRY            1
WOLCOTT   

In [55]:
staffing_df.reset_index()["City"].unique()

array(['ANDOVER', 'ANSONIA', 'ASHFORD', 'AVON', 'BALTIC', 'BARKHAMSTED',
       'BERLIN', 'BETHANY', 'BETHEL', 'BLOOMFIELD', 'BOLTON', 'BOZRAH',
       'BRANFORD', 'BRIDGEPORT', 'BRISTOL', 'BROOKFIELD', 'BROOKLYN',
       'CANAAN', 'CANTERBURY', 'CHAPLIN', 'CHESHIRE', 'CLINTON',
       'COLCHESTER', 'COLEBROOK', 'COLLINSVILLE', 'COLUMBIA', 'CORNWALL',
       'COVENTRY', 'CROMWELL', 'DANBURY', 'DANIELSON', 'DARIEN',
       'DEEP RIVER', 'DERBY', 'EAST GRANBY', 'EAST HAMPTON',
       'EAST HARTFORD', 'EAST HARTLAND', 'EAST HAVEN', 'EAST LYME',
       'EAST WINDSOR', 'EASTFORD', 'EASTON', 'ELLINGTON', 'ENFIELD',
       'FAIRFIELD', 'FALLS VILLAGE', 'FARMINGTON', 'GLASTONBURY',
       'GRANBY', 'GREENWICH', 'GRISWOLD', 'GUILFORD', 'HAMDEN', 'HAMPTON',
       'HARTFORD', 'HEBRON', 'KENT', 'LAKEVILLE', 'LEBANON', 'LEDYARD',
       'LISBON', 'LITCHFIELD', 'MADISON', 'MANCHESTER', 'MARLBOROUGH',
       'MERIDEN', 'MIDDLETOWN', 'MILFORD', 'MONROE', 'MOODUS', 'MYSTIC',
       'NAUGATUCK', 'NEW B